<a href="https://colab.research.google.com/github/lachlandeer/ruokay-tweets-analysis/blob/main/berttopic_ruokay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct 31 14:34:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## Set up Data pull from Google Compute

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.cloud import storage


In [5]:
# Initialize Google Cloud Storage client
client = storage.Client()
bucket_name = 'ruokay'
bucket = client.get_bucket(bucket_name)

In [7]:
import pandas as pd

In [8]:
source_blob_name = 'out_data/ruokay_tweets_28_days_2011_2019.csv'  # File path in the bucket
local_file_path = '/tmp/ruokay_tweets_28_days_2011_2019.csv'  # Path to save locally in Colab

# Download the file from the bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(source_blob_name)
blob.download_to_filename(local_file_path)

print(f"File downloaded to {local_file_path}")

# Load the CSV into a pandas DataFrame
df = pd.read_csv(local_file_path)

# Display the first few rows of the DataFrame
df.head()

File downloaded to /tmp/ruokay_tweets_28_days_2011_2019.csv


<ipython-input-8-e2debc51b3d1>:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_file_path)


,id,created_at,text,lang,year,second_thursday,days_from_thursday
0,107321434191298561,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...,en,2011.0,2011-09-08 00:00:00+10:00,-12.0
1,107318405278859265,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,en,2011.0,2011-09-08 00:00:00+10:00,-12.0
2,107318045436936193,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,en,2011.0,2011-09-08 00:00:00+10:00,-12.0
3,107316121253855232,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...,en,2011.0,2011-09-08 00:00:00+10:00,-12.0
4,107299839867494400,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...,en,2011.0,2011-09-08 00:00:00+10:00,-12.0


## Import packages for NLP

In [10]:
!pip install tweet-preprocessor


In [11]:
!pip install bertopic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.2 MB/s eta 0:00:00


In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# import tweet preprocessor
import preprocessor as p

import re
# import pandas as pd
from nltk.corpus import stopwords
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [13]:
data = df.filter(['created_at','text'], axis=1)


In [14]:
data.head()

,created_at,text
0,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...
1,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...
2,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...
3,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...
4,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...


In [15]:
# Customizing the preprocessor to exclude removing hashtags since they are valuable as a rich information
p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION,p.OPT.SMILEY,p.OPT.NUMBER )

#forming a separate feature for cleaned tweets
data['text_clean'] = data['text'].apply(lambda x: p.clean(str(x)) if isinstance(x, str) else "")

data.head()

,created_at,text,text_clean
0,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...,RT : RT : In a disaster send a text message to...
1,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,RT : It's go time! Game day & looks a picture ...
2,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,RT : It's go time! Game day & looks a picture ...
3,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...,It's go time! Game day & looks a picture for t...
4,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...,there's nothing stupid about ruok imok (whethe...


In [16]:
# Function to remove only the '#' symbol but keep the word
def remove_hashtag_sign(text):
    # This will match '#' only at the beginning of words
    text = re.sub(r'#(?=\w)', '', text)
    return text

# Apply the function to the 'text_clean' column
data['text_clean'] = data['text_clean'].apply(remove_hashtag_sign)

In [17]:
#Remove extra white spaces, punctuation and apply lower casing
data['text_clean'] = data['text_clean'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
data.head()

,created_at,text,text_clean
0,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...,rt : rt : in a disaster send a text message to...
1,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,rt : it's go time! game day & looks a picture ...
2,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,rt : it's go time! game day & looks a picture ...
3,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...,it's go time! game day & looks a picture for t...
4,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...,there's nothing stupid about ruok imok (whethe...


In [18]:
# Replace all instances of "rt" and ":" with an empty string
data['text_clean'] = data['text_clean'].str.replace(r'\brt\b|:', '', regex=True)

# Remove excess whitespace and strip leading/trailing whitespace
data['text_clean'] = data['text_clean'].str.replace(r'\s+', ' ', regex=True).str.strip()

data.head()

,created_at,text,text_clean
0,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...,in a disaster send a text message to your love...
1,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,it's go time! game day & looks a picture for t...
2,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,it's go time! game day & looks a picture for t...
3,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...,it's go time! game day & looks a picture for t...
4,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...,there's nothing stupid about ruok imok (whethe...


In [19]:
data = data[data['text_clean'].str.strip() != ""]
data.head()

,created_at,text,text_clean
0,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...,in a disaster send a text message to your love...
1,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,it's go time! game day & looks a picture for t...
2,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,it's go time! game day & looks a picture for t...
3,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...,it's go time! game day & looks a picture for t...
4,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...,there's nothing stupid about ruok imok (whethe...


In [20]:
data.shape

(413476, 3)

In [21]:
# test on a smaller sample
sampled_data = data.sample(n=10000, random_state=42)  # Set random_state for reproducibility


In [23]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
from nltk.corpus import stopwords

# Prepare the tweet list
tweet_list = data['text_clean'].tolist()

# Remove English stopwords using NLTK
stop_words = stopwords.words('english')  # Renamed variable to avoid confusion

# Initialize CountVectorizer with stopwords
vectorizer_model = CountVectorizer(stop_words=stop_words)

In [31]:
topic_model = BERTopic(embedding_model="vinai/bertweet-base",
                       vectorizer_model=vectorizer_model,
                       #low_memory=True,
                       calculate_probabilities=True,
                       verbose=True,
                       n_gram_range=(1, 3)
                       )


In [ ]:
topics, probs = topic_model.fit_transform(tweet_list)


2024-10-31 14:55:05,680 - BERTopic - Embedding - Transforming documents to embeddings.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Batches:   0%|          | 0/12922 [00:00<?, ?it/s]

2024-10-31 15:08:42,833 - BERTopic - Embedding - Completed ✓
2024-10-31 15:08:42,835 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-31 15:27:37,726 - BERTopic - Dimensionality - Completed ✓
2024-10-31 15:27:37,736 - BERTopic - Cluster - Start clustering the reduced embeddings


In [27]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3746,-1_suicide_mental_someone_ok,"[suicide, mental, someone, ok, health, today, ...",[it's a simple question to ask but the convers...
1,0,344,0_offering_thanks_calls_well,"[offering, thanks, calls, well, great, cute, f...",[proud offering free calls for . start a conve...
2,1,291,1_national_reminder_needed_could,"[national, reminder, needed, could, gut, actio...","[it's ruokday, our national day of action and ..."
3,2,284,2_four_steps_use_question,"[four, steps, use, question, needed, tips, wan...",[want to start an ruok conversation? use these...
4,3,252,3_may_like_think_us,"[may, like, think, us, struggling, people, man...",[today is r u ok? day is our national day of a...
...,...,...,...,...,...
157,156,11,156_dosomething_left_look_ru,"[dosomething, left, look, ru, right, save, one...",[look to your left. look to your right. ask - ...
158,157,11,157_dont_expert_fix_workmate,"[dont, expert, fix, workmate, judge, problems,...",[you dont have to fix a mates problems on ruok...
159,158,10,158_saving_achieve_stigmawatch_centre,"[saving, achieve, stigmawatch, centre, http, a...",[need advice on mentalillness suicide stories?...
160,159,10,159_tasty_ca_dinner_nice,"[tasty, ca, dinner, nice, walk, look, sure, ta...",[today is world mental health day. be sure you...


In [28]:
# coherence
docs = tweet_list
# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]

topic_words = [[words for words, _ in topic_model.get_topic(topic) if words!='']
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

0.6987508006064107

In [29]:
topic_model.visualize_hierarchy()
